In [2]:
!pip install sentence_transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=da2eaed7d02274ccdbe2f6c52f9f40c6e1b6f1efeae2d8c5f44389eb72bee2ff
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [15]:
import pandas as pd
from copy import deepcopy
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm

In [358]:
chart_dept = pd.read_csv("/content/drive/MyDrive/Achatbot/data/chart_dept_for_title.csv")

In [359]:
chart_dept_for_title = chart_dept[['TITLE_CHART', 'CYCTIME', 'SERVICE_NAME']]
chart_dept_for_title = chart_dept_for_title.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
chart_dept_for_title.head()

,TITLE_CHART,CYCTIME,SERVICE_NAME
0,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết)
1,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết)
2,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết)
3,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,4g lũy kế
4,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,3g tăng giảm


In [360]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2',
                            cache_folder="/content/drive/MyDrive/Achatbot/model_dir")

In [361]:
def cosinesimilarity(vector1, vector2):
    cosine = np.dot(vector1, vector2)/(norm(vector1)*norm(vector2))
    return cosine

def remove_chitieu(title_chart, service_name, modell):
    features = title_chart.split("_")
    output2 = deepcopy(features)
    modell.eval()
    service_name='dtdv'
    embedding1 = modell.encode(service_name)
    ranking = {}
    for feature in features:
        score = cosinesimilarity(modell.encode(feature), embedding1)
        ranking[feature]=score

    output = max(ranking, key=ranking.get)
    output2.remove(output)
    return output, ranking, output2

In [362]:
def remove_time(title_chart_rm_donvi):
    features = title_chart_rm_donvi.split("_")
    if len(features) == 1:
        features = "_".join(features)
        return features, ""
    output2 = ""
    output = deepcopy(features)
    for feature in features:
        for i in ['ngay', 'thang', 'nam', 'quy', 'ngày', 'tháng', 'năm', 'quý']:
            if i in feature:
                output2 = feature
                output.remove(feature)
                break

    output = "_".join(output)
    return output, output2

In [363]:
def remove_donvi(title_chart):
    features = title_chart.split("_")
    if " " not in features[0] and features[0] not in ["company", 'info', 'test', 'rank', 'rada', 'table', 'tooltip', '21', 'dgsdf', 'khdn', 'dt-cp-ln', 'arpu', 'mou', 'dou']:
        output =  features[0]
        output2 = "_".join(features[1:])
    elif features[0] in ['tập đoàn', 'ban cntt', 'khối tỉnh/tp', 'công đoàn', 'viettel tỉnh/tp']:
        output = features[0]
        output2 = "_".join(features[1:])
    else:
        output = ""
        output2 = "_".join(features[0:])
    return output, output2


In [367]:
chart_dept_for_title['DONVI_FEATURE'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_donvi(x)[0])
chart_dept_for_title['TITLE_CHART_RM_DONVI'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_donvi(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[0])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[1])
chart_dept_for_title.drop(columns=['TITLE_CHART_RM_DONVI'])[:20]

,TITLE_CHART,CYCTIME,SERVICE_NAME,DONVI_FEATURE,TITLE_CHART_RM_DONVI_TIME,TIME_FEATURE
0,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),,doanh thu dịch vụ tổng,
1,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),,doanh thu dịch vụ tổng,
2,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),,doanh thu dịch vụ tổng,
3,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,4g lũy kế,stg,tb register di động_theo hạ tầng (lk hệ thống),
4,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,3g tăng giảm,stg,tb register di động_theo hạ tầng (lk hệ thống),
5,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,3g lũy kế,stg,tb register di động_theo hạ tầng (lk hệ thống),
6,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,3g tăng/giảm,stg,tb register di động_theo hạ tầng (lk hệ thống),
7,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,2g lũy kế,stg,tb register di động_theo hạ tầng (lk hệ thống),
8,stg_tb register di động_theo hạ tầng (lk hệ th...,thang,2g tăng/giảm,stg,tb register di động_theo hạ tầng (lk hệ thống),
9,thuê bao di động psc thực,ngay,thuê bao thực tăng thêm,,thuê bao di động psc thực,


In [ ]:
chart_dept_for_title['TITLE_CHART_RM_TIME'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_time(x)[0])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_time(x)[1])

In [320]:
len(list(set(chart_dept_for_title['don_vi'])))

120

In [72]:
i=0
chart_temp = chart_dept_for_title[chart_dept_for_title['SERVICE_NAME']=='doanh thu dịch vụ (chi tiết)'][20*i:20*(i+1)]
chart_temp

,TITLE_CHART,CYCTIME,SERVICE_NAME,TITLE_CHART_RM_TIME,TIME_FEATURE
0,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),doanh thu dịch vụ tổng,
1,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),doanh thu dịch vụ tổng,
2,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),doanh thu dịch vụ tổng,
26,gli_dtdv_từng quý,quy,doanh thu dịch vụ (chi tiết),gli_dtdv,từng quý
32,hbh_dtdv_từng quý,quy,doanh thu dịch vụ (chi tiết),hbh_dtdv,từng quý
33,dng_dtdv_từng tháng,thang,doanh thu dịch vụ (chi tiết),dng_dtdv,từng tháng
34,dng_dtdv_từng năm,nam,doanh thu dịch vụ (chi tiết),dng_dtdv,từng năm
93,hbh_dtdv_từng tháng,thang,doanh thu dịch vụ (chi tiết),hbh_dtdv,từng tháng
94,hbh_dtdv chi nhánh,thang,doanh thu dịch vụ (chi tiết),hbh_dtdv chi nhánh,
98,hbh_dtdv_từng năm,nam,doanh thu dịch vụ (chi tiết),hbh_dtdv,từng năm


In [80]:
service_name = list(set(chart_dept_for_title['SERVICE_NAME']))[1]
# print(len(list(set(chart_dept_for_title['SERVICE_NAME']))))
print(service_name)
chart_dept_for_title[chart_dept_for_title['SERVICE_NAME']==service_name]['TITLE_CHART'].unique()

số lượng pa/kn xử lý trong thời gian quy định (trong hạn) (viettelpay)


array(['tỷ lệ xử lý đúng hạn dv viettelpay_bo_lũy kế ngày (điều kiện >= kpi)',
       'tỷ lệ xử lý đúng hạn dv viettelpay_bo_từng tháng (điều kiện >= kpi)'],
      dtype=object)

In [42]:
chart_temp['TITLE_CHART_RM_TIME_CHITIEU'] = chart_temp[['TITLE_CHART_RM_TIME', 'SERVICE_NAME']].apply(lambda x: remove_chitieu(x[0], x[1], model)[2], axis=1)
chart_temp['CHITIEU_FEATURE'] = chart_temp[['TITLE_CHART_RM_TIME', 'SERVICE_NAME']].apply(lambda x: remove_chitieu(x[0], x[1], model)[0], axis=1)
chart_temp['RANKING'] = chart_temp[['TITLE_CHART_RM_TIME', 'SERVICE_NAME']].apply(lambda x: remove_chitieu(x[0], x[1], model)[1], axis=1)

In [ ]:
a = {
    'doanh thu dịch vụ (chi tiết)': ['dtdv', 'dt dịch vụ', 'doanh thu dịch vụ'],

}

In [43]:
chart_temp

,TITLE_CHART,CYCTIME,SERVICE_NAME,TITLE_CHART_RM_TIME,TIME_FEATURE,TITLE_CHART_RM_TIME_CHITIEU,CHITIEU_FEATURE,RANKING
0,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),doanh thu dịch vụ tổng,,[],doanh thu dịch vụ tổng,{'doanh thu dịch vụ tổng': 0.1843297}
1,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),doanh thu dịch vụ tổng,,[],doanh thu dịch vụ tổng,{'doanh thu dịch vụ tổng': 0.1843297}
2,doanh thu dịch vụ tổng,ngay,doanh thu dịch vụ (chi tiết),doanh thu dịch vụ tổng,,[],doanh thu dịch vụ tổng,{'doanh thu dịch vụ tổng': 0.1843297}
26,gli_dtdv_từng quý,quy,doanh thu dịch vụ (chi tiết),gli_dtdv,từng quý,[gli],dtdv,"{'gli': 0.7500777, 'dtdv': 0.99999994}"
32,hbh_dtdv_từng quý,quy,doanh thu dịch vụ (chi tiết),hbh_dtdv,từng quý,[hbh],dtdv,"{'hbh': 0.7359503, 'dtdv': 0.99999994}"
33,dng_dtdv_từng tháng,thang,doanh thu dịch vụ (chi tiết),dng_dtdv,từng tháng,[dng],dtdv,"{'dng': 0.8354447, 'dtdv': 0.99999994}"
34,dng_dtdv_từng năm,nam,doanh thu dịch vụ (chi tiết),dng_dtdv,từng năm,[dng],dtdv,"{'dng': 0.8354447, 'dtdv': 0.99999994}"
93,hbh_dtdv_từng tháng,thang,doanh thu dịch vụ (chi tiết),hbh_dtdv,từng tháng,[hbh],dtdv,"{'hbh': 0.7359503, 'dtdv': 0.99999994}"
94,hbh_dtdv chi nhánh,thang,doanh thu dịch vụ (chi tiết),hbh_dtdv chi nhánh,,[dtdv chi nhánh],hbh,"{'hbh': 0.7359503, 'dtdv chi nhánh': 0.7197545}"
98,hbh_dtdv_từng năm,nam,doanh thu dịch vụ (chi tiết),hbh_dtdv,từng năm,[hbh],dtdv,"{'hbh': 0.7359503, 'dtdv': 0.99999994}"
